In [20]:
import logging
import time
import random
from datetime import datetime

n_campi = 'CIGR'

data_e_hora_atuais = datetime.now()
data_e_hora = data_e_hora_atuais.strftime('%Y-%m-%d_%H-%M')

logger = logging.getLogger('consolidação')
logger.setLevel(logging.DEBUG)

fHandler = logging.FileHandler(f'logs/{data_e_hora_atuais}_consolidacao.log', mode='w')
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

fHandler.setFormatter(formatter)
logger.addHandler(fHandler)

# logging.basicConfig(level=logging.DEBUG, filename='app.log', filemode='x', format='%(asctime)s - %(levelname)s - %(message)s')
logger.info(f'Iniciando consolidação campus {n_campi}')
# logger.debug('debug message')
# logger.warning('warn message')

In [21]:
import pandas as pd

logger.info(f'carregando palnilha de levantamento fisico campus {n_campi}')
levantamento = pd.read_excel('consolidacao/levantamento/2023_levantamento_CIGR.xlsx')

logger.info(f'carregando palnilha de estoque suap campus {n_campi}')
suap = pd.read_excel('consolidacao/suap/2023-08-11_15-21_CIGR_relatorio_almox.xlsx')

logger.info(f'tratamdp dados da planilha de levantameto fisico {n_campi}')

# removendo itens nulos e deixando apenas os retificados
retificacao = levantamento[levantamento['Retificação'].str.contains('Retificação').fillna(False)]


# lista de itens vencidos
# vencido = levantamento[levantamento['Material Vencido - MV'].str.contains('x').fillna(False)]
# lista de itens não identificados
# nao_identificado = levantamento[levantamento['Sem identificação - SI'].str.contains('x').fillna(False)]

# print(levantamento.info())
# print(suap.info())

# planilha contendo apenas itens válidos
levantamento_ret = levantamento.drop_duplicates(subset=['Código'], keep='last')
levantamento_ret = levantamento_ret.rename(columns={'Código': 'codigo'})

# elemento despesa do levantamento
ed_lev = levantamento_ret['codigo'].str[3:5]


# adicionando ed ao data frame geral
# levantamento_ret['ED'] = ed_lev

logger.info(f'removendo dados duplicados')
# lista de servidores
servidores = levantamento['Endereço de e-mail'].drop_duplicates(keep='last')
# reindexando uutilizando list comprehension
servidores.index = [x for x in range(0, len(servidores))]

logger.info(f'cruzando dados entre levantamento fisico e suap')

levantamento_cross = levantamento_ret.drop(columns=['Material de pouca movimentação - PM', 'Material Vencido - MV',
                                                 'Material próximo ao vencimento - PV', 'Material danificado ou avariado - MD',
                                                 'Material Obsoleto - MO', 'Sem identificação - SI', 'Valor unitário',
                                                 'Campus', 'Retificação', 'Carimbo de data/hora'])
levantamento_cross = levantamento_cross.rename(columns={'Endereço de e-mail': 'servidor', 'Nome': 'descricao_lev',
                                                        'Quantidade': 'qtd_lev'})
levantamento_cross = levantamento_cross[['codigo', 'descricao_lev', 'qtd_lev', 'servidor']]

suap_cross = suap.rename(columns={'descricao': 'descricao_suap', 'qtd': 'qtd_suap'})
suap_cross = suap_cross.drop(columns=['vlr_total', 'vlr_unt'])
# display(suap_cross)
# display(levantamento_cross)

logger.info(f'consolidando dados')

logger.info(f'exportando planilha de consolidação')
consolidacao = pd.merge(suap_cross, levantamento_cross, on='codigo', how='outer', indicator=True)
consolidacao.to_excel(f'consolidacao/{n_campi}_consolidacao.xlsx', index=False)

logger.info(f'consolidação campus {n_campi} finalizada')

In [22]:
logger.info(f'montando planilha de ajustes ')

consolidacao['diferenca'] = consolidacao['qtd_suap'] - consolidacao['qtd_lev']
display(consolidacao)

pedidos = consolidacao[consolidacao['_merge'].str.contains('both')]
pedidos = pedidos[['codigo','ed' ,'descricao_suap', 'qtd_suap', 'qtd_lev', 'diferenca']]

acao = []
for valor in pedidos['diferenca']:
    
    if valor > 0:
        baixa = "pedido no suap"
        acao.append(baixa)
    elif valor < 0:
        baixa = "saida sem pedido"
        acao.append(baixa)
    else:
        baixa = 'adequado'
        acao.append(baixa)

logger.info(f'exportando planilha de ajustes ')

pedidos['acao'] = acao
pedidos.to_excel(f'consolidacao/pedidos/{n_campi}_pedidos.xlsx', index=False)
display(pedidos)

,codigo,descricao_suap,und,qtd_suap,ed,descricao_lev,qtd_lev,servidor,_merge,diferenca
0,30.11.922,"Álcool 70. agente desinfetante, Líquido, límpi...",BOMBONA,9,11 - MATERIAL QUIMICO,"Álcool 70. agente desinfetante, Líquido, límpi...",9,cmpsg@igarassu.ifpe.edu.br,both,0
1,30.14.177,"BOLA DE VÔLEI APLICAÇÃO EM (PRAIA),CIRC. 65 À ...",UN,4,14 - MATERIAL EDUCATIVO E ESPORTIVO,"BOLA DE VÔLEI APLICAÇÃO EM (PRAIA),CIRC. 65 À ...",4,edivonaldo.araujo@igarassu.ifpe.edu.br,both,0
2,30.14.24,BOMBA DE ENCHER BOLA,UN,3,14 - MATERIAL EDUCATIVO E ESPORTIVO,BOMBA DE ENCHER BOLA,4,cmpsg@igarassu.ifpe.edu.br,both,-1
3,30.14.278,COLETE ESPORTIVO POLIESTER.,UN,100,14 - MATERIAL EDUCATIVO E ESPORTIVO,COLETE ESPORTIVO POLIESTER.,100,cmpsg@igarassu.ifpe.edu.br,both,0
4,30.14.283,"COLETE ESPORTIVO, MATERIAL POLIÉSTER, NUMERADO...",UN,24,14 - MATERIAL EDUCATIVO E ESPORTIVO,"COLETE ESPORTIVO, MATERIAL POLIÉSTER, NUMERADO...",24,cmpsg@igarassu.ifpe.edu.br,both,0
...,...,...,...,...,...,...,...,...,...,...
156,30.36.523,"SABONETEIRA, ALTURA 28,70 CM, LARGURA 12,50 CM...",UN,50,36 - MATERIAL HOSPITALAR,"SABONETEIRA, ALTURA 28,70 CM, LARGURA 12,50 CM...",50,cmpsg@igarassu.ifpe.edu.br,both,0
157,30.36.532,KIT DE EMERGÊNCIA PARA DESCONTAMINAÇÃO DE AMBI...,Conjunto,1,36 - MATERIAL HOSPITALAR,KIT DE EMERGÊNCIA PARA DESCONTAMINAÇÃO DE AMBI...,1,cmpsg@igarassu.ifpe.edu.br,both,0
158,30.42.877,"JOGO DE CHAVES ALLEN COM 11 PEÇAS, 1,5MM, 2MM,...",JOGO,1,42 - FERRAMENTAS,"JOGO DE CHAVES ALLEN COM 11 PEÇAS, 1,5MM, 2MM,...",1,edivonaldo.araujo@igarassu.ifpe.edu.br,both,0
159,30.44.61,PLACA SETA INDICATIVA DE AGUA PRESSURIZADA (AP).,UN,2,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS,PLACA SETA INDICATIVA DE AGUA PRESSURIZADA (AP).,4,cmpsg@igarassu.ifpe.edu.br,both,-2


,codigo,ed,descricao_suap,qtd_suap,qtd_lev,diferenca,acao
0,30.11.922,11 - MATERIAL QUIMICO,"Álcool 70. agente desinfetante, Líquido, límpi...",9,9,0,adequado
1,30.14.177,14 - MATERIAL EDUCATIVO E ESPORTIVO,"BOLA DE VÔLEI APLICAÇÃO EM (PRAIA),CIRC. 65 À ...",4,4,0,adequado
2,30.14.24,14 - MATERIAL EDUCATIVO E ESPORTIVO,BOMBA DE ENCHER BOLA,3,4,-1,saida sem pedido
3,30.14.278,14 - MATERIAL EDUCATIVO E ESPORTIVO,COLETE ESPORTIVO POLIESTER.,100,100,0,adequado
4,30.14.283,14 - MATERIAL EDUCATIVO E ESPORTIVO,"COLETE ESPORTIVO, MATERIAL POLIÉSTER, NUMERADO...",24,24,0,adequado
...,...,...,...,...,...,...,...
156,30.36.523,36 - MATERIAL HOSPITALAR,"SABONETEIRA, ALTURA 28,70 CM, LARGURA 12,50 CM...",50,50,0,adequado
157,30.36.532,36 - MATERIAL HOSPITALAR,KIT DE EMERGÊNCIA PARA DESCONTAMINAÇÃO DE AMBI...,1,1,0,adequado
158,30.42.877,42 - FERRAMENTAS,"JOGO DE CHAVES ALLEN COM 11 PEÇAS, 1,5MM, 2MM,...",1,1,0,adequado
159,30.44.61,44 - MATERIAL DE SINALIZACAO VISUAL E OUTROS,PLACA SETA INDICATIVA DE AGUA PRESSURIZADA (AP).,2,4,-2,saida sem pedido
